<a href="https://colab.research.google.com/github/praveensurya2808/B2b-Courier-charges-analysis/blob/main/Courier_charges_for_B2B_company.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
order_report = pd.read_csv('Order Report.csv')
invoice = pd.read_csv('Invoice.csv')
sku_master = pd.read_csv('SKU Master.csv')
pincodes = pd.read_csv('pincodes.csv')
courier_company_rates = pd.read_csv('Courier Company - Rates.csv')
print("Order Report:")
print(order_report.head())
print("\nSKU Master:")
print(sku_master.head())
print("\nPincode Mapping:")
print(pincodes.head())
print("\nCourier Invoice:")
print(invoice.head())
print("\nCourier Company rates:")
print(courier_company_rates.head())



Order Report:
   ExternOrderNo            SKU  Order Qty  Unnamed: 3  Unnamed: 4
0     2001827036  8904223818706        1.0         NaN         NaN
1     2001827036  8904223819093        1.0         NaN         NaN
2     2001827036  8904223819109        1.0         NaN         NaN
3     2001827036  8904223818430        1.0         NaN         NaN
4     2001827036  8904223819277        1.0         NaN         NaN

SKU Master:
             SKU  Weight (g)  Unnamed: 2  Unnamed: 3  Unnamed: 4
0  8904223815682         210         NaN         NaN         NaN
1  8904223815859         165         NaN         NaN         NaN
2  8904223815866         113         NaN         NaN         NaN
3  8904223815873          65         NaN         NaN         NaN
4  8904223816214         120         NaN         NaN         NaN

Pincode Mapping:
   Warehouse Pincode  Customer Pincode Zone  Unnamed: 3  Unnamed: 4
0             121003            507101    d         NaN         NaN
1             121003       

In [6]:
# Check for missing values
print("\nMissing values in Website Order Report:")
print(order_report.isnull().sum())
print("\nMissing values in SKU Master:")
print(sku_master.isnull().sum())
print("\nMissing values in Pincode Mapping:")
print(pincodes.isnull().sum())
print("\nMissing values in Courier Invoice:")
print(invoice.isnull().sum())
print("\nMissing values in courier company rates:")
print(courier_company_rates.isnull().sum())


Missing values in Website Order Report:
ExternOrderNo      0
SKU                0
Order Qty          0
Unnamed: 3       400
Unnamed: 4       400
dtype: int64

Missing values in SKU Master:
SKU            0
Weight (g)     0
Unnamed: 2    66
Unnamed: 3    66
Unnamed: 4    66
dtype: int64

Missing values in Pincode Mapping:
Warehouse Pincode      0
Customer Pincode       0
Zone                   0
Unnamed: 3           124
Unnamed: 4           124
dtype: int64

Missing values in Courier Invoice:
AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64

Missing values in courier company rates:
fwd_a_fixed         0
fwd_a_additional    0
fwd_b_fixed         0
fwd_b_additional    0
fwd_c_fixed         0
fwd_c_additional    0
fwd_d_fixed         0
fwd_d_additional    0
fwd_e_fixed         0
fwd_e_additional    0
rto_a_fixed        

In [7]:
order_report = order_report.drop(columns=['Unnamed: 3','Unnamed: 4' ])
sku_master = sku_master.drop(columns=['Unnamed: 3','Unnamed: 4', 'Unnamed: 2'])
pincodes = pincodes.drop(columns=['Unnamed: 3','Unnamed: 4'])
print(pincodes.head())

   Warehouse Pincode  Customer Pincode Zone
0             121003            507101    d
1             121003            486886    d
2             121003            532484    d
3             121003            143001    b
4             121003            515591    d


In [8]:
merged_data = pd.merge(order_report, sku_master, on='SKU')
print(merged_data.head())

   ExternOrderNo            SKU  Order Qty  Weight (g)
0     2001827036  8904223818706        1.0         127
1     2001827036  8904223819093        1.0         150
2     2001827036  8904223819109        1.0         100
3     2001827036  8904223818430        1.0         165
4     2001827036  8904223819277        1.0         350


In [20]:
merged_data = merged_data.rename(columns={'ExternOrderNo' : 'Order ID'})

In [10]:
ss_courier = pincodes.drop_duplicates(subset=['Customer Pincode'])
courier_ss = invoice[['Order ID','Customer Pincode','Type of Shipment']]
pincodes= courier_ss.merge(ss_courier,on='Customer Pincode')
print(pincodes.head())

     Order ID  Customer Pincode Type of Shipment  Warehouse Pincode Zone
0  2001806232            507101  Forward charges             121003    d
1  2001806273            486886  Forward charges             121003    d
2  2001806408            532484  Forward charges             121003    d
3  2001806458            143001  Forward charges             121003    b
4  2001807012            515591  Forward charges             121003    d


In [31]:
merged_2 = merged_data.merge(pincodes,on='Order ID')
merged_2.head()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone Charged by Courier Company
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e


In [32]:
merged_2['Weights (Kgs)'] = merged_2['Weight (g)'] / 1000

In [33]:
def weight_slab(weight):
  i = round(weight % 1,1)
  if i == 0.0:
    return weight
  if i >= 0.5 :
    return int(weight) + weight + 1.0
  else:
    return int(weight) + 0.5
merged_2['Weights (Kgs)']= (merged_2['Weights (Kgs)']).apply(weight_slab)
invoice['Weight Slab Charged by Courier Company']=(invoice['Charged Weight']).apply(weight_slab)

In [34]:
pincodes = pincodes.rename(columns={'Zone' : 'Delivery Zone Charged by Courier Company'})
merged_2 = merged_2.rename(columns={'Weights (Kgs)':'Weight Slab As Per SS'})
merged_2 = merged_2.rename(columns={'Delivery Zone Charged by Courier Company': 'Delivery Zone As Per SS'})
merged_2.head()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone As Per SS,Weight Slab As Per SS
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.5
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.5
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.5
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.5
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.5


In [35]:
total_Expected_charge = []
for _, row in merged_2.iterrows():
  fwd_category = 'fwd_'+ row['Delivery Zone As Per SS']
  fwd_fixed = courier_company_rates.at[0,fwd_category+'_fixed']
  fwd_additional = courier_company_rates.at[0,fwd_category+'_additional']
  rto_category = 'rto_'+row['Delivery Zone As Per SS']
  rto_fixed = courier_company_rates.at[0, rto_category + '_fixed']
  rto_additional = courier_company_rates.at[0,rto_category+'_additional']
  weight_slab = row['Weight Slab As Per SS']

  if row['Type of Shipment'] == 'Forward charges':
    additional_weight = max(0, (weight_slab - 0.5) / 0.5)
    total_Expected_charge.append(fwd_fixed + additional_weight * fwd_additional)
  elif row['Type of Shipment'] == 'Forward and RTO charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_Expected_charge.append(fwd_fixed + additional_weight * (fwd_additional + rto_additional))
  else:
      total_Expected_charge.append(0)

merged_2['Expected Charge as per ABC'] = total_Expected_charge
print(merged_2.head())

     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode  \
0  2001827036  8904223818706        1.0         127            173213   
1  2001827036  8904223819093        1.0         150            173213   
2  2001827036  8904223819109        1.0         100            173213   
3  2001827036  8904223818430        1.0         165            173213   
4  2001827036  8904223819277        1.0         350            173213   

  Type of Shipment  Warehouse Pincode Delivery Zone As Per SS  \
0  Forward charges             121003                       e   
1  Forward charges             121003                       e   
2  Forward charges             121003                       e   
3  Forward charges             121003                       e   
4  Forward charges             121003                       e   

   Weight Slab As Per SS  Expected Charge as per ABC  
0                    0.5                        56.6  
1                    0.5                        56.6  
2    

In [36]:
merged_output = merged_2.merge(invoice,on='Order ID')
print(merged_output.head())


     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode_x  \
0  2001827036  8904223818706        1.0         127              173213   
1  2001827036  8904223819093        1.0         150              173213   
2  2001827036  8904223819109        1.0         100              173213   
3  2001827036  8904223818430        1.0         165              173213   
4  2001827036  8904223819277        1.0         350              173213   

  Type of Shipment_x  Warehouse Pincode_x Delivery Zone As Per SS  \
0    Forward charges               121003                       e   
1    Forward charges               121003                       e   
2    Forward charges               121003                       e   
3    Forward charges               121003                       e   
4    Forward charges               121003                       e   

   Weight Slab As Per SS  Expected Charge as per ABC       AWB Code  \
0                    0.5                        56.6  109112241

In [37]:
df_diff = merged_output
df_diff['Difference'] = df_diff['Billing Amount (Rs.)'] - df_diff['Expected Charge as per ABC']
df_new = df_diff[['Order ID', 'Difference', 'Expected Charge as per ABC']]

print(df_new.head())

     Order ID  Difference  Expected Charge as per ABC
0  2001827036        61.3                        56.6
1  2001827036        61.3                        56.6
2  2001827036        61.3                        56.6
3  2001827036        61.3                        56.6
4  2001827036        61.3                        56.6


In [38]:
#caclulate the total order in each category
total_correctly_charged = len(df_new[df_new['Difference'] == 0])
total_over_charged = len(df_new[df_new['Difference'] > 0])
total_under_charged = len(df_new[df_new['Difference'] < 0])

# calculate the amount in each category
amount_correctly_charged = df_new[df_new['Difference'] == 0]['Expected Charge as per ABC'].sum()
amount_over_charged = df_new[df_new['Difference'] > 0]['Difference'].sum()
amount_less_charged = df_new[df_new['Difference'] < 0]['Difference'].sum()

summary_data = {'Description': ['Total order of SS courier correctly charged', 'Total order of SS courier over charged', 'Total order of SS courier under charged'],
                'count':[total_correctly_charged, total_over_charged, total_under_charged],
                'Amount': [amount_correctly_charged, amount_over_charged, amount_less_charged]}
df_summary= pd.DataFrame(summary_data)
print(df_summary)



                                   Description  count      Amount
0  Total order of SS courier correctly charged     14    732.2000
1       Total order of SS courier over charged    378  33036.6468
2      Total order of SS courier under charged      9   -264.6000


In [41]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Pie(labels=df_summary['Description'],
                            values=df_summary['count'],
                            textinfo='label+percent',
                            hole=0.4))
fig.update_layout(title_text='Total order of SS courier charged')
fig.show()